# ETL Project


## Extraction from 2 CSV files

In [33]:
#import required dependencies here:
import pandas as pd
import datetime
from sqlalchemy import create_engine
#from config import db_password

In [34]:
#Read all_years_o3.csv into df1 and display the first 5 rows
df1 = pd.read_csv('./Resources/all_years_o3.csv')
df1.head()



,Date,Country,City,Specie,count,min (ppb),max (ppb),median (ppb)
0,2017-01-01,US,Los Angeles,o3,24,2.0,31.0,10.0
1,2017-01-01,CN,Shanghai,o3,188,1.0,39.0,11.0
2,2017-01-01,US,Manhattan,o3,24,3.0,28.0,16.0
3,2017-01-01,US,Jacksonville,o3,46,2.0,18.0,12.0
4,2017-01-01,CN,Beijing,o3,235,1.0,8.0,2.0


In [95]:
#Read all_years_pm25.csv into df2 and display the first 5 rows
df2 = pd.read_csv('./Resources/all_years_pm25.csv')
df2.head()

,Date,Country,City,Specie,count,min (ug/m3),max (ug/m3),median (ug/m3)
0,2017-01-01,IN,New Delhi,pm25,24,54.9,282.7,177.7
1,2017-01-01,CN,Shanghai,pm25,215,12.0,63.2,34.4
2,2017-01-01,US,Jacksonville,pm25,115,3.1,113.6,8.1
3,2017-01-01,US,Los Angeles,pm25,69,3.3,109.7,15.9
4,2017-01-01,US,Manhattan,pm25,96,4.3,23.0,12.1


# Transform

In [96]:
#drop last 3 columns on df1 and display the first 5 rows.
#df1 = df1.drop(df1.columns[-3:], axis= 1) # droping last three columns 
#df1 = df1.drop(df1.index[:5]) # dropping first five rows
df1.head()


,Date,Country,City,Count_o3
0,2017-01-01,US,Los Angeles,24
1,2017-01-01,CN,Shanghai,188
2,2017-01-01,US,Manhattan,24
3,2017-01-01,US,Jacksonville,46
4,2017-01-01,CN,Beijing,235


In [94]:
#rename the count columns in df1 to 'Count_o3' and show first 5rows
df1.rename(columns={'count': 'Count_o3'}, inplace= True)
df1.head()


,Date,Country,City,Count_o3
0,2017-01-01,US,Los Angeles,24
1,2017-01-01,CN,Shanghai,188
2,2017-01-01,US,Manhattan,24
3,2017-01-01,US,Jacksonville,46
4,2017-01-01,CN,Beijing,235


In [56]:
#delete specie in df1 and whow first 5rows
df1['Specie'] = df1['Specie']= "" # emptying the column 
df1 = df1.drop(['Specie'], axis= 1)
df1.head()


,Date,Country,City,Count_o3
0,2017-01-01,US,Los Angeles,24
1,2017-01-01,CN,Shanghai,188
2,2017-01-01,US,Manhattan,24
3,2017-01-01,US,Jacksonville,46
4,2017-01-01,CN,Beijing,235


In [60]:
#drop last 3 columns on df2 and display the first 5 rows.
df2 = df2.drop(df2.columns[-3:], axis= 1)
df2.head()


,Date,Country,City,Specie,count
0,2017-01-01,IN,New Delhi,pm25,24
1,2017-01-01,CN,Shanghai,pm25,215
2,2017-01-01,US,Jacksonville,pm25,115
3,2017-01-01,US,Los Angeles,pm25,69
4,2017-01-01,US,Manhattan,pm25,96


In [61]:
#rename the count columns in df2 to 'Count_o3' and show first 5rows
df2.rename(columns={'count': 'Count_o3'}, inplace= True)
df2.head()

,Date,Country,City,Specie,Count_o3
0,2017-01-01,IN,New Delhi,pm25,24
1,2017-01-01,CN,Shanghai,pm25,215
2,2017-01-01,US,Jacksonville,pm25,115
3,2017-01-01,US,Los Angeles,pm25,69
4,2017-01-01,US,Manhattan,pm25,96


In [62]:
#delete specie in df2 and whow first 5rows
df2 = df2.drop(['Specie'], axis = 1)
df2.head()


,Date,Country,City,Count_o3
0,2017-01-01,IN,New Delhi,24
1,2017-01-01,CN,Shanghai,215
2,2017-01-01,US,Jacksonville,115
3,2017-01-01,US,Los Angeles,69
4,2017-01-01,US,Manhattan,96


In [67]:
#merge the two dfs into one on date, country, city and show the first 5rows
dfs = pd.concat([df1, df2])
dfs.head()

,Date,Country,City,Count_o3
0,2017-01-01,US,Los Angeles,24
1,2017-01-01,CN,Shanghai,188
2,2017-01-01,US,Manhattan,24
3,2017-01-01,US,Jacksonville,46
4,2017-01-01,CN,Beijing,235


In [1]:
# Alternative way to concatenate
#df = df1.merge(df2, on=['Date', 'Country', 'City'], how = "inner")
#new_df = pd.merge(df1,df2, how='left', left_on = ['Date', 'Country', 'City'], right =['Date', 'Country', 'City'])

# Load

# Schema Table

```sql
CREATE TABLE merge_counts (
    "Date" DATE,
    "Country" VARCHAR,
    "City" VARCHAR,
    "Count_o3" INT,
    "Count_pm25" INT
);
```

In [88]:
from config import username, password
from sqlalchemy import create_engine, engine


In [90]:
from sqlalchemy import create_engine, engine
#SQL login info
# Add the password to postgres
username = 'postgres'
password = 'postgres'
protocol = "postgresql"
url = "localhost"
port = 5432 # ur port might be 5432
db = "postgres"
connection_string = f"{protocol}://{username}:{password}@{url}:{port}/{db}"
engine = create_engine(connection_string)

In [93]:
# Confirm tables 
engine.table_names()

AttributeError: 'Engine' object has no attribute 'table_names'

In [ ]:
# Load my pandas dataframe
new_df.to_sql(name="merge_counts", con=engine, if_exists='append', index=False)

In [ ]:
# Confirm data has been added to the table
# Query the data inside the merge_counts table


,Date,Country,City,Count_o3,Count_pm25
0,2017-01-01,US,Los Angeles,24,69
1,2017-01-01,CN,Shanghai,188,215
2,2017-01-01,US,Manhattan,24,96
3,2017-01-01,US,Jacksonville,46,115
4,2017-01-01,CN,Beijing,235,386
